# Import libraries, load drive and define path/files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %cd "/content/drive/MyDrive/Model for Automated Portfolio Optimization"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
import random
import pickle

In [ ]:
path = "/content/drive/MyDrive/Model for Automated Portfolio Optimization"
model_file_path = path + "/model"
data = path + "/data/amibroker_all_data.txt"
data_path = path
dataset_path = path + "/Wave Dataset"

In [ ]:
STOCK_TICKER = 0
DATA = 1
TIME = 1
OPEN = 2
HIGH = 3
LOW = 4
CLOSE = 5
VOLUME = 6

def get_history(input):
  return input[1]

def get_time(input):
  return input[0]

def get_high(input):
  return input[1]

def get_low(input):
  return input[2]

def get_close(input):
  return input[3]

def get_volume(input):
  return input[4]

def make_history_block(date_time_ymd, open_price, high_price, low_price, close_price, volume):
  return [date_time_ymd, float(open_price), float(high_price), float(low_price), float(close_price), float(volume)]

In [ ]:
with open(data) as stock_data:
    lines = stock_data.read().splitlines()

for i in range(len(lines)):
    lines[i] = lines[i].split(sep=',')

In [ ]:
data = dict()
for i in range(len(lines)):
  data[lines[i][STOCK_TICKER]] = [None, []]

for i in range(len(lines)):
  if lines[i][STOCK_TICKER] != "<Ticker>":
    data[lines[i][STOCK_TICKER]][STOCK_TICKER] = lines[i][STOCK_TICKER]
    data[lines[i][STOCK_TICKER]][DATA].append(make_history_block(lines[i][TIME],
                                                                 lines[i][OPEN],
                                                                 lines[i][HIGH],
                                                                 lines[i][LOW],
                                                                 lines[i][CLOSE],
                                                                 lines[i][VOLUME]))

In [ ]:
lst_vn30 = ["MBB", "MSN", "VCB", "ACB", "BID", "PNJ",
            "GAS", "TCB", "CTG", "VJC", "PLX", "BVH",
            "SAB", "STB", "MWG", "PDR", "POW", "GVR",
            "VRE", "NVL", "KDH", "TPB", "VPB", "VNM",
            "HDB", "SSI", "FPT", "VHM", "VIC", "HPG"]

dataset = []

def make_data():
    for i in lst_vn30:
        timeline = [data[i][DATA][x][OPEN:VOLUME] for x in range(0, len(data[i][DATA]))] 
        dataset.append(timeline)
    return

In [ ]:
make_data()

In [ ]:
def divide_list(lst1, lst2):
  ret = []
  for i in range(0, len(lst1)):
    # print(lst1[i], lst2[i])
    if (lst1[i] == 0) and (lst2[i] == 0):
      ret.append(-1)
      continue
    ret.append(2 * (lst1[i]/(lst1[i] + lst2[i]) - 0.5))
  return ret

wave_data = []
for i in range(0, len(dataset)):
  temp_data = []
  for j in range(1, len(dataset[i])):
    # print(i, j)
    temp_data.append(divide_list(dataset[i][j], dataset[i][j-1]))
  wave_data.append(temp_data)

# Define simple linear regression wave extractor

In [ ]:
from sklearn.linear_model import LinearRegression
import math 
import matplotlib.pyplot as plt

def input_wave(stock_index, start_time, length): # stock_index: example: #0 = MBB, ..., start_time, length, return 4 linear regression slopes [OPEN, HIGH, LOW, VOLUME]
  if len(dataset[stock_index]) < abs(start_time) + length:
    return [0, 0, 0, 0]
  x = [[i] for i in range(0, length)]
  y_open = []
  y_high = []
  y_low = []
  y_volume = []
  for i in range(0, length):
    y_open.append([dataset[stock_index][start_time + i][0]])
    y_high.append([dataset[stock_index][start_time + i][1]])
    y_low.append([dataset[stock_index][start_time + i][2]])
    y_volume.append([dataset[stock_index][start_time + i][3]])
  # 
  # plt.plot(x, y_volume)
  # print(max(y_volume)[0]/min(y_volume)[0])
  x = np.asarray(x)
  y_open = np.asarray(y_open)
  y_high = np.asarray(y_high)
  y_low = np.asarray(y_low)
  y_volume = np.asarray(y_volume)
  model_open = LinearRegression().fit(x, y_open)
  model_high = LinearRegression().fit(x, y_high)
  model_low = LinearRegression().fit(x, y_low)
  model_volume = LinearRegression().fit(x, y_volume)
  # print(model_volume.intercept_)
  return [math.atan(model_open.coef_[0][0]/model_open.intercept_[0]),
          math.atan(model_high.coef_[0][0]/model_high.intercept_[0]),
          math.atan(model_low.coef_[0][0]/model_low.intercept_[0]),
          math.atan(model_volume.coef_[0][0]/model_volume.intercept_[0]),
          ]
  

# Define control model

## Reward function

To build a reward function

Input:
+ Start time (in form of how many days before present, negative number)
+ Finish time (present?) / Length
+ Portfolio

Output:
+ Return
+ Largest lost


In [ ]:
def get_stock_value(i, time):
  if i == len(dataset):
    return 1
  if abs(time) >= len(dataset[i]):
    return 1
  return dataset[i][time][0]

def portfolio_to_value(dataset, time, portfolio):
  value = 0
  for i in range(len(dataset)):
    value += get_stock_value(i, time) * portfolio[i]
  return value + portfolio[-1]
  
def reward(start_time, length, portfolio_distribution):
  initial_money = 1000000
  portfolio = [0 for _ in range(len(dataset) + 1)]
  for i in range(len(portfolio)):
    portfolio[i] = initial_money * portfolio_distribution[i] / get_stock_value(i, start_time)
  # print(portfolio)
  max_money = initial_money
  max_risk = 0
  current_money = initial_money
  for i in range(1, length):
    current_money = portfolio_to_value(dataset, start_time + i, portfolio)
    max_risk = max(max_money - current_money, max_risk) 
  return (current_money, max_risk)

## Make data point for reinforcement training

In [ ]:
portfolio_dummy = [random.randint(1, 1000) for i in range(31)]

def softmax(x):
  x = np.asarray(x)
  return np.exp(x) / np.sum(np.exp(x), axis=0)

def de_softmax(x):
  return np.log(x) - np.average(np.log(x), axis=0)

def prob(x):
  x = np.asarray(x)
  return x / np.sum(x, axis=0)

def add_noise(x):
  x = np.asarray(x)
  x = x + np.random.uniform(-0.1, 0.1, x.shape)
  return x

def generate_random_probability():
  portfolio_dummy = [random.randint(1, 1000) for i in range(31)]
  return prob(portfolio_dummy)

def generate_noise_probability(x):
  return softmax(add_noise(de_softmax(x)))

[0.03319248 0.0462582  0.00400311 0.05504281 0.03675081 0.04047593
 0.04731458 0.01145335 0.05242967 0.04030913 0.04520182 0.02123874
 0.01434449 0.05170688 0.04520182 0.0185144  0.02190593 0.01050817
 0.04375625 0.0094518  0.02724341 0.00544868 0.04692539 0.02846659
 0.04431224 0.02924497 0.05059491 0.02023796 0.03814078 0.01328811
 0.04703658]
[0.03480496 0.04270491 0.00373152 0.05088461 0.03727878 0.04330701
 0.0445309  0.01125096 0.05132974 0.04276034 0.04669048 0.01996824
 0.01455923 0.05023457 0.04967827 0.01832846 0.02383502 0.01134332
 0.04138012 0.01042423 0.02537612 0.00605083 0.05049842 0.02930615
 0.04211602 0.03146379 0.04628629 0.02125746 0.03749794 0.01467835
 0.04644298]
[0.03127237 0.04774845 0.00407071 0.0550825  0.03409283 0.04094477
 0.04974884 0.01201536 0.05028735 0.03759672 0.04192042 0.0206522
 0.01352539 0.04784819 0.05075891 0.01964917 0.02041024 0.0103605
 0.0451563  0.00925685 0.0272191  0.00544828 0.04442491 0.03180886
 0.04755112 0.02755897 0.04696631 0.01

In [ ]:
def make_data_point(limit_time, length=750):
  len_time = max([len(dataset[i]) for i in range(len(dataset))])
  start_time = random.randint(-len_time, limit_time - length)
  portfolio = generate_random_probability()
  portfolio_noise = generate_noise_probability(portfolio)
  # print(portfolio)
  # print(portfolio_noise)
  exist = [0 for _ in range(len(dataset))]
  exist.append(1)
  for i in range(len(dataset)):
    if len(dataset[i]) > abs(start_time):
      exist[i] = 1
  for i in range(len(exist)):
    portfolio[i] = portfolio[i] * exist[i]
    portfolio_noise[i] = portfolio_noise[i] * exist[i]
  portfolio = prob(portfolio)
  portfolio_noise = prob(portfolio_noise)
  reward_portfolio = reward(start_time, length, portfolio)
  reward_portfolio_noise = reward(start_time, length, portfolio_noise)
  feature = []
  for i in range(0, len(dataset)):
    temp = []
    if exist[i] != 0:
      for j in range(2, 8):
        temp.append([input_wave(i, start_time - 2**j + 1, 2**j), 1])
    else:
      for j in range(2, 8):
        temp.append([[0, 0, 0, 0], 0])
    feature.append(temp)
  # print(start_time, -len_time, limit_time - length)
  # print(portfolio)
  # print(portfolio_noise)
  # print(feature)
  return reward_portfolio, reward_portfolio_noise, portfolio, portfolio_noise, feature

In [ ]:
for cnt in range(6, 100):
  temp_list = []
  for i in range(0, 10000):
    temp = make_data_point(-750)
    temp_list.append(temp)

  with open(dataset_path + "/" + "dataset" + str(cnt) + ".pickle", "wb") as fp:
    pickle.dump(temp_list, fp)

Output hidden; open in https://colab.research.google.com to view.